In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [43]:
df='/content/gdrive/MyDrive/Humans.zip'

In [ ]:
df1=zipfile.ZipFile(df,'r')

In [ ]:
df1.extractall('/content/gdrive/MyDrive/Humans')

In [26]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [27]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
train_datagen = ImageDataGenerator(rescale=1/255)

In [30]:
train_generator = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Humans',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 7219 images belonging to 1 classes.


In [31]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=25,
      verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/25
8/8 [==============================] - 50s 6s/step - loss: 0.0887 - acc: 0.8750
Epoch 2/25
8/8 [==============================] - 36s 4s/step - loss: 3.8152e-07 - acc: 1.0000
Epoch 3/25
8/8 [==============================] - 33s 4s/step - loss: 2.4420e-08 - acc: 1.0000
Epoch 4/25
8/8 [==============================] - 33s 4s/step - loss: 1.9701e-08 - acc: 1.0000
Epoch 5/25
8/8 [==============================] - 34s 4s/step - loss: 5.8718e-08 - acc: 1.0000
Epoch 6/25
8/8 [==============================] - 32s 4s/step - loss: 1.8975e-08 - acc: 1.0000
Epoch 7/25
8/8 [==============================] - 32s 4s/step - loss: 1.8590e-08 - acc: 1.0000
Epoch 8/25
8/8 [==============================] - 32s 4s/step - loss: 2.4685e-08 - acc: 1.0000
Epoch 9/25
4/8 [==============>...............] - ETA: 20s - loss: 8.6001e-09 - acc: 1.0000

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


8/8 [==============================] - 36s 5s/step - loss: 4.5265e-09 - acc: 1.0000
Epoch 10/25
8/8 [==============================] - 32s 4s/step - loss: 5.8525e-08 - acc: 1.0000
Epoch 11/25
8/8 [==============================] - 33s 4s/step - loss: 5.7272e-10 - acc: 1.0000
Epoch 12/25
8/8 [==============================] - 33s 4s/step - loss: 7.8332e-11 - acc: 1.0000
Epoch 13/25
8/8 [==============================] - 33s 4s/step - loss: 7.8234e-10 - acc: 1.0000
Epoch 14/25
8/8 [==============================] - 33s 4s/step - loss: 1.0338e-10 - acc: 1.0000
Epoch 15/25
8/8 [==============================] - 33s 4s/step - loss: 1.2526e-10 - acc: 1.0000
Epoch 16/25
8/8 [==============================] - 33s 4s/step - loss: 8.2382e-10 - acc: 1.0000
Epoch 17/25
8/8 [==============================] - 35s 4s/step - loss: 4.5393e-10 - acc: 1.0000
Epoch 18/25
8/8 [==============================] - 32s 4s/step - loss: 2.8532e-10 - acc: 1.0000
Epoch 19/25
8/8 [==============================] - 3

In [32]:
model.save('/content/gdrive/MyDrive/face_model2.h5')

In [34]:
from tensorflow.keras.applications.vgg16 import VGG16

In [35]:
base_model=VGG16(input_shape=(300,300,3),     #Shaoe of our images
                 include_top=False,           #Leave out the last fully connected layer
                 weights='imagenet')

58900480/58889256 [==============================] - 1s 0us/step


In [36]:
from keras import layers

In [37]:
for layer in base_model.layers:
  layer.trainable=False

In [38]:
#Flatten the output latyer to one dimension
x = layers.Flatten()(base_model.output)

#Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

#Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

#Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)


In [39]:
model=tf.keras.models.Model(base_model.input, x)

In [40]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy',metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [44]:
history=model.fit(train_generator, steps_per_epoch=8, epochs=10)

Epoch 1/10
8/8 [==============================] - 243s 30s/step - loss: 7.1566e-11 - acc: 1.0000
Epoch 2/10
8/8 [==============================] - 237s 29s/step - loss: 3.7662e-10 - acc: 1.0000
Epoch 3/10
8/8 [==============================] - 236s 29s/step - loss: 6.9733e-11 - acc: 1.0000
Epoch 4/10
8/8 [==============================] - 244s 30s/step - loss: 3.8479e-11 - acc: 1.0000
Epoch 5/10
8/8 [==============================] - 239s 30s/step - loss: 3.1308e-09 - acc: 1.0000
Epoch 6/10
8/8 [==============================] - 241s 30s/step - loss: 7.8340e-11 - acc: 1.0000
Epoch 7/10
8/8 [==============================] - 237s 29s/step - loss: 1.5736e-10 - acc: 1.0000
Epoch 8/10
8/8 [==============================] - 236s 29s/step - loss: 8.3459e-11 - acc: 1.0000
Epoch 9/10
8/8 [==============================] - 240s 30s/step - loss: 2.7908e-11 - acc: 1.0000
Epoch 10/10
8/8 [==============================] - 237s 30s/step - loss: 7.9231e-11 - acc: 1.0000


In [45]:
model.save('/content/gdrive/MyDrive/vgg_model.h5')